### Maxwell A. Fine 2024-07-15

This notebook is for running `ddplan.py` from `presto` and making functions to implement it into the pipeline (`check_frb.py`)


### from the `presto` tutorial:

DDplan.py determines near-optimal ways to de-disperse your data to
maintain sensitivity to fast pulsars yet save CPU and I/O time

* Assumes using prepsubband to do multiple-passes through the data
using “subband” de-dispersion

* Specify command line information from readfile, or (New!) give the
filename and DDplan.py will determine the observation details

* The new “-w” option will write out a dedisp*py file that you can run to
dedisperse your data (and edit as needed, i.e. to add rfifind masks)



### Notes

- It looks `like DDplan.py` is smart enough to read in the dt, numchan, blocklen, BW, fctr from the `.fil` files




### `DDplan.py` options
```
DDplan.py 

    usage:  DDplan.py [options] [raw PSRFITS or filterbank file]
  [-h, --help]                    : Display this help
  [-o outfile, --outfile=outfile] : Output .eps plot file (default is xwin)
  [-l loDM, --loDM=loDM]          : Low DM to search   (default = 0 pc cm-3)
  [-d hiDM, --hiDM=HIDM]          : High DM to search  (default = 1000 pc cm-3)
  [-f fctr, --fctr=fctr]          : Center frequency   (default = 1400MHz)
  [-b BW, --bw=bandwidth]         : Bandwidth in MHz   (default = 300MHz)
  [-n #chan, --numchan=#chan]     : Number of channels (default = 1024)
  [-k blocklen, --blocklen=#spec] : Spectra per subint (for downsampling) (default = 1024)
  [-c cDM, --cohdm=cDM]           : Coherent DM in each chan  (default = 0.0)
  [-t dt, --dt=dt]                : Sample time (s)    (default = 0.000064 s)
  [-s #subbands, --subbands=nsub] : Number of subbands (default = #chan)
  [-p #procs, --procs=nprocs]     : # CPUs dedispersing for mpiprepsubband (default = 1)
  [-r resolution, --res=res]      : Acceptable time resolution (ms)
  [-w, --write]                   : Write a dedisp.py file for the plan

  The program generates a good plan for de-dispersing raw data.  It
  trades a small amount of sensitivity in order to save computation costs.
  It will determine the observation parameters from the raw data file
  if it exists.
```


In [15]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os.path import basename, dirname
import glob
# we will be running presto in a docker image, so we will execute presto commands using subprocess
import subprocess 
import os

# Define the container name and docker image
container_name = "ddplanv2"
docker_image = "c7897abba927"

local_path = "/home/afinemax/afinemax/khazad-dum/research/astron_2024/ddplan"
container_path = "/"




### Attempt with Docker

In [21]:
# Check if the container already exists
def container_exists(name):
    result = subprocess.run(["docker", "ps", "-a", "--filter", f"name={name}", "--format", "{{.Names}}"], capture_output=True, text=True)
    return name in result.stdout


if container_exists(container_name):
    subprocess.run(["docker", "restart", container_name])
else:
    subprocess.run([
        "docker", "run", "-d", "--name", container_name,
        "-v", f"{local_path}:{container_path}",
        "-e", f"DISPLAY={os.environ['DISPLAY']}",
        "-v", "/tmp/.X11-unix:/tmp/.X11-unix",
        docker_image, "sleep", "infinity"
    ])

# Function to run a command inside the Docker container
def run_command_in_container(command):
    '''Runs command in the docker image cmd line, cmd is a single str'''
    result = subprocess.run(["docker", "exec", container_name, "bash", "-c", command], capture_output=True, text=True)
    return result.stdout, result.stderr

# Example commands to run inside the container
stdout, stderr = run_command_in_container('echo Hello from inside the container!')
print(stdout)


stdout, stderr = run_command_in_container('pwd')
print('pwd', stdout)


stdout, stderr = run_command_in_container('ls')
print('ls', stdout)

stdout, stderr = run_command_in_container('ls /data')
print('ls /data', stdout)






ddplan
Hello from inside the container!

pwd /software

ls presto5
tempo

ls /data 2dDBscan_final_v1.png
DBscan_test001.png
LICENSE
README.md
astron_research_2024.yml
bandpass_corr
calling_fetch
dbscan_clustering
ddplan
diagnostic_plots
example_pipeline__h5_output
flow_charts
frb_example_data_june_2024
important_docs_and_papers
inject_sims_into_fil
my_scripts
noise_channels
presto_and_tut
ref_cmds.md
running_presto_with_docker
target_visilbility



In [31]:
# Command I want is 

file= '/data/ddplan/FRB20240209A_L2_Band_2024_07_11_14_33_31.fil'

central_dm = 176
low_dm =  0.9* central_dm
high_dm = 1.1 * central_dm

command = 'DDplan.py ' + file + ' -d ' + str(high_dm) +' -l ' + low_dm +'' -n 320 -b 100  -t 0.0001984 -f 1350.16'

stdout, stderr = run_command_in_container(command)

print(stdout)




Using:
        dt = 0.0001984 s
   numchan = 320
  blocklen = 2400
        BW = 100 MHz
      fctr = 1350.16 MHz
from '/data/ddplan/FRB20240209A_L2_Band_2024_07_11_14_33_31.fil'


Minimum total smearing     : 0.281 ms
--------------------------------------------
Minimum channel smearing   : 0 ms
Minimum smearing across BW : 0.00169 ms
Minimum sample time        : 0.198 ms

Setting the new 'best' resolution to : 0.198 ms
Best guess for optimal initial dDM is 1.177



### Notes:
- Docker wont work for us because `PGPLOT` errors out before we get the optimal search params

### Just trying on my local laptop with a subprocess call

### Notes:
- we need to put the full `DDplan.py -d 200 -l 160 -n 320 -b 100  -t 0.0001984 -f 1350.16' garb, not just low and high dm
- it doesn't error out, but it will use the default for low and high dm 

In [81]:
import subprocess
import presto.filterbank as fil
import numpy as np


def dd_plan(fil_file, numchan, central_dm quiet=False, low_dm = 0.9, high_dm = 1.1):
    """
    Calculate DM (Dispersion Measure) search parameters using DDplan.py output.

    Parameters:
    ----------
    fil_file : str
        Path to the filterbank file (.fil) containing header information.
    numchan : int
        Number of frequency channels in the data.
    central_dm : float, 
        Central DM (Dispersion Measure) value around which to calculate search parameters.
    quiet : bool, optional
        If True, suppresses printing of intermediate status messages. Default is False.
    low_dm : float, optional
        Lower bound of the DM range for searching. Default is 0.9 times central_dm.
    high_dm : float, optional
        Upper bound of the DM range for searching. Default is 1.1 times central_dm.

    Returns:
    -------
    tuple of floats
        Tuple containing the calculated parameters:
        - dDM (float): Dispersion Measure step size suggested by DDplan.py.
        - n_DMS (int): Number of DM steps suggested by DDplan.py.
        - low_dm (float): Lower DM bound used in the calculation.
        - high_dm (float): Upper DM bound used in the calculation.

    Notes:
    ------
    This function reads header information from a filterbank file, computes necessary bandwidth (BW) and center frequency (fctr) values,
    and runs DDplan.py with these parameters via a subprocess call. It parses the output of DDplan.py to extract dDM, n_DMS, low_dm, and high_dm values,
    which are then returned.
    """
    try:
        hdr, hdr_size = fil.read_header(fil_file)
        dt = hdr['tsamp']
        #numchan = hdr['nchans']
        BW = np.fabs(hdr['foff']) * numchan
        fctr = hdr['fch1'] + 0.5 * hdr['foff'] * numchan - 0.5 * hdr['foff']

        low_dm = 0.9 * central_dm
        high_dm = 1.1 * central_dm

        command = f'DDplan.py -d {high_dm} -l {low_dm} -n {numchan} -b {BW} -t {dt} -f {fctr}'
        if not quiet:
            print('Running', command)

        with subprocess.Popen(command, stdout=subprocess.PIPE, bufsize=1, text=True, shell=True) as p:
            dDM, n_DMS, low_dm, high_dm = None, None, low_dm, high_dm  # Initialize variables

            for line in p.stdout:
                if not quiet:
                    print(line, end='')

                # Check for lines starting with 'Low DM' and parse values
                elif line.startswith('  Low DM'):
                    try:
                        values_line = next(p.stdout).strip()
                        parts = values_line.split()
                        low_dm = float(parts[0])
                        high_dm = float(parts[1])
                        dDM = float(parts[2])
                        n_DMS = int(parts[4])
                    except ValueError as ve:
                        print(f"ValueError encountered while parsing line: {values_line}. Error: {ve}")
                    except StopIteration:
                        print("Reached end of output while expecting more data.")
                    except Exception as e:
                        print(f"An unexpected error occurred: {e}")

            return dDM, n_DMS, low_dm, high_dm

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, None, None
    

    

# Usage
fil_file = 'FRB20240209A_L2_Band_2024_07_11_14_33_31.fil'
dDM, n_DMS, low_dm, high_dm = dd_plan(fil_file, central_dm=1750)

print('dDM, n_DMS, low_dm, high_dm')
print(dDM, n_DMS, low_dm, high_dm)


Running DDplan.py -d 1925.0000000000002 -l 1575.0 -n 320 -b 100.0 -t 0.0001984 -f 1350.15625

Minimum total smearing     : 1.68 ms
--------------------------------------------
Minimum channel smearing   : 1.66 ms
Minimum smearing across BW : 0.00169 ms
Minimum sample time        : 0.198 ms

Setting the new 'best' resolution to : 1.66 ms
   Note: min_chan_smearing > dt (i.e. data is higher resolution than needed)
         New dt is 8 x 0.1984 ms = 1.5872 ms
Best guess for optimal initial dDM is 9.415

  Low DM    High DM     dDM  DownSamp   #DMs  WorkFract
 1575.000   1935.000   10.00       8      36   1



dDM, n_DMS, low_dm, high_dm
None None 1575.0 1925.0000000000002


In [66]:
import subprocess
import presto.filterbank as fil
import numpy as np


def dd_plan(fil_file, numchan, central_dm quiet=False, low_dm = 0.9, high_dm = 1.1):
    """
    Calculate DM (Dispersion Measure) search parameters using DDplan.py output.

    Parameters:
    ----------
    fil_file : str
        Path to the filterbank file (.fil) containing header information.
    numchan : int
        Number of frequency channels in the data.
    central_dm : float, 
        Central DM (Dispersion Measure) value around which to calculate search parameters.
    quiet : bool, optional
        If True, suppresses printing of intermediate status messages. Default is False.
    low_dm : float, optional
        Lower bound of the DM range for searching. Default is 0.9 times central_dm.
    high_dm : float, optional
        Upper bound of the DM range for searching. Default is 1.1 times central_dm.

    Returns:
    -------
    tuple of floats
        Tuple containing the calculated parameters:
        - dDM (float): Dispersion Measure step size suggested by DDplan.py.
        - n_DMS (int): Number of DM steps suggested by DDplan.py.
        - low_dm (float): Lower DM bound used in the calculation.
        - high_dm (float): Upper DM bound used in the calculation.

    Notes:
    ------
    This function reads header information from a filterbank file, computes necessary bandwidth (BW) and center frequency (fctr) values,
    and runs DDplan.py with these parameters via a subprocess call. It parses the output of DDplan.py to extract dDM, n_DMS, low_dm, and high_dm values,
    which are then returned.
    """
    try:
        hdr, hdr_size = fil.read_header(fil_file)
        dt = hdr['tsamp']
        #numchan = hdr['nchans']
        BW = np.fabs(hdr['foff']) * numchan
        fctr = hdr['fch1'] + 0.5 * hdr['foff'] * numchan - 0.5 * hdr['foff']

        low_dm = 0.9 * central_dm
        high_dm = 1.1 * central_dm

        command = f'DDplan.py -d {high_dm} -l {low_dm} -n {numchan} -b {BW} -t {dt} -f {fctr}'
        if not quiet:
            print('Running', command)

        with subprocess.Popen(command, stdout=subprocess.PIPE, bufsize=1, text=True, shell=True) as p:
            dDM, n_DMS, low_dm, high_dm = None, None, low_dm, high_dm  # Initialize variables

            for line in p.stdout:
                if not quiet:
                    print(line, end='')

                # Check for lines starting with 'Low DM' and parse values
                elif line.startswith('  Low DM'):
                    try:
                        values_line = next(p.stdout).strip()
                        parts = values_line.split()
                        low_dm = float(parts[0])
                        high_dm = float(parts[1])
                        dDM = float(parts[2])
                        n_DMS = int(parts[4])
                    except ValueError as ve:
                        print(f"ValueError encountered while parsing line: {values_line}. Error: {ve}")
                    except StopIteration:
                        print("Reached end of output while expecting more data.")
                    except Exception as e:
                        print(f"An unexpected error occurred: {e}")

            return dDM, n_DMS, low_dm, high_dm

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, None, None
    

    

# Usage
fil_file = 'FRB20240209A_L2_Band_2024_07_11_14_33_31.fil'
dDM, n_DMS, low_dm, high_dm = dd_plan(fil_file, central_dm=1750)

print('dDM, n_DMS, low_dm, high_dm')
print(dDM, n_DMS, low_dm, high_dm)


ddout  ddplan  ddplan.ipynb  FRB20240209A_L2_Band_2024_07_11_14_33_31.fil





```DDplan.py FRB20240209A_L2_Band_2024_07_11_14_33_31.fil -d 550.0 -l 450.0 -n 320 -b 100.0 -t 0.0001984 -f 1350.15625

Using:
        dt = 0.0001984 s
   numchan = 320
  blocklen = 2400
        BW = 100 MHz
      fctr = 1350.16 MHz
from 'FRB20240209A_L2_Band_2024_07_11_14_33_31.fil'


Minimum total smearing     : 0.281 ms
--------------------------------------------
Minimum channel smearing   : 0 ms
Minimum smearing across BW : 0.00169 ms
Minimum sample time        : 0.198 ms

Setting the new 'best' resolution to : 0.198 ms
Best guess for optimal initial dDM is 1.177

  Low DM    High DM     dDM  DownSamp   #DMs  WorkFract
    0.000    622.000    1.00       1     622   0.8681
  622.000   1000.000    2.00       2     189   0.1319





```DDplan.py FRB20240209A_L2_Band_2024_07_11_14_33_31.fil -d 1650.0000000000002 -l 1350.0 -n 320 -b 100.0 -t 0.0001984 -f 1350.15625

Using:
        dt = 0.0001984 s
   numchan = 320
  blocklen = 2400
        BW = 100 MHz
      fctr = 1350.16 MHz
from 'FRB20240209A_L2_Band_2024_07_11_14_33_31.fil'


Minimum total smearing     : 0.281 ms
--------------------------------------------
Minimum channel smearing   : 0 ms
Minimum smearing across BW : 0.00169 ms
Minimum sample time        : 0.198 ms

Setting the new 'best' resolution to : 0.198 ms
Best guess for optimal initial dDM is 1.177

  Low DM    High DM     dDM  DownSamp   #DMs  WorkFract
    0.000    622.000    1.00       1     622   0.8681
  622.000   1000.000    2.00       2     189   0.1319
```